# TP: Introduction à Pytorch

## Partie A : exercices 
### Exercice 1 : Manipulation de Tenseurs

**Objectif :**  
Se familiariser avec les dimensions des tenseurs et les opérations matricielles sous **PyTorch**.

1. Créer un tenseur **A** de taille `(5, 3)` contenant des valeurs aléatoires tirées d’une loi normale.
2. Créer un tenseur **B** de taille `(3, 2)` rempli de uns.
3. Calculer le produit matriciel **C = A × B**.
4. Afficher la dimension (*shape*) du tenseur **C**.
5. Transposer le tenseur **C** et expliquer l’impact de cette opération sur la mémoire.




### Exercice 2: 

1. Créer un tenseur nommé `mon_tenseur` contenant une séquence d’entiers de 0 à 23 inclus, en utilisant la fonction `arange`.

2. Redimensionner ce tenseur afin qu’il prenne la forme de 4 lignes et 6 colonnes.

3. Afficher :
   - la taille du tenseur,
   - le type de données associé.

4. Multiplier l’ensemble des valeurs du tenseur par **10**, puis afficher le résultat final.




### Exercice 3 : Calcul de Gradient avec Autograd

Soit la fonction scalaire suivante :

\[
y = 3x^2 + 2x + 1
\]

1. Définir un tenseur `x = 2.0` en activant le suivi du gradient (`requires_grad=True`).
2. Implémenter l’équation de `y`.
3. Utiliser la méthode `.backward()` pour calculer les gradients.
4. Extraire et afficher la valeur de \(\frac{dy}{dx}\) stockée dans `x.grad`.




### Exercice 4 : Architecture de Réseau de Neurones

En utilisant le module **torch.nn**, créer une classe `SimpleNet` :

1. La couche d’entrée doit accepter **10 caractéristiques**.
2. Une couche cachée de **5 neurones** avec une activation **ReLU**.
3. Une couche de sortie de **1 neurone** (régression).
4. Instancier le modèle et effectuer un **forward pass** sur un vecteur aléatoire.

---

## Partie B:
**Classification d'images avec le dataset CIFAR-10**

Le dataset CIFAR-10 contient 60 000 images réparties sur 10 classes différentes, chacune avec 6 000 images de taille 32x32 pixels.
Ces classes sont : avions, automobiles, oiseaux, chats, cerfs, chiens, grenouilles, chevaux, navires et camions.

`classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')`

L'objectif de ce TP est de mettre en place un modèle de classification d'images dans l'environnement PyTorch.

Plan du TP
- 1) Charger les données CIFAR-10 avec PyTorch.
- 2) Préparer les données.
- 3) Créer un modèle simple de réseau de neurones convolutionnel (CNN).
- 4) Entraîner ce modèle sur les données CIFAR-10.
- 5) Évaluer la performance du modèle
